In [26]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [27]:
def sorted_tuple(x):
    return tuple(sorted(x))

In [28]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/drop-mini-validation").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
next(iter(experiments), None)

97 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/drop-musique-mini',
   'name': 'default',
   'split': 'train'},
  'qa': {'model': 'llama-3-8b',
   'temperature': 0.0,
   'top_p': 0.999,
   'system_prompt': 'direct/format-few.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'manual/direct.json',
   'n_shot': 0,
   'n_sc': 1},
  'run': 1},
 'metrics': {'exact_match': 0.37,
  'f1': 0.5993370884585718,
  'fuzzy_match': 0.41,
  '0hops': {'exact_match': 0.37,
   'f1': 0.5993370884585718,
   'fuzzy_match': 0.41},
  'gen_token_count': {'all': {'count': 100.0,
    'mean': 6.41,
    'std': 4.431510219810265,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 7.0,
    'max': 28.0},
   'success': {'count': 65.0,
    'mean': 5.3076923076923075,
    'std': 2.8718628368789054,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 5.0,
    'max': 22.0},
   'fail': {'count': 35.0,
    'mean': 8.457142857142857,
    'std

In [29]:
df = pd.json_normalize(experiments)
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)
print(f"{len(df)} experiments before preprocessing")
df.head()

97 experiments before preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.top_p,params.qa.system_prompt,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
0,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,0.999,direct/format-few.txt,...,5.00,22.0,35.0,8.457143,5.917784,3.0,5.00,6.0,9.00,28.0
1,cb4b874538a73c5774648d414aaddcf69cac56cb,3472a49ed1fb84d89cb1bd950d2f2b2a18c2ae01,rapid-zoon,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,ccot/format-thought-drop.txt,...,286.25,404.0,38.0,246.421053,143.691605,128.0,179.75,209.0,257.50,873.0
2,cb4b874538a73c5774648d414aaddcf69cac56cb,456d70902afc97c9c50d14a66844a3b68bae2bf3,splay-dook,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,4.00,6.0,54.0,6.462963,8.608274,1.0,2.00,3.0,6.75,34.0
3,cb4b874538a73c5774648d414aaddcf69cac56cb,bc72f60fa17d88fcb165fbac7ef813c4b370395d,non-U-shes,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,4.00,41.0,49.0,4.571429,7.302967,1.0,2.00,2.0,4.00,48.0
4,cb4b874538a73c5774648d414aaddcf69cac56cb,e551b58e5ae14d9f8fb622825fcd534e37c5b1a1,oaten-loup,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,29.00,94.0,52.0,35.423077,58.662994,9.0,14.00,24.0,29.00,366.0


In [30]:
if 'params.qa.top_p' not in df.columns:
    df['params.qa.top_p'] = 0.999

df['params.qa.top_p'] = df['params.qa.top_p'].map(lambda x: 0.999 if x > 0.99 else x)

In [31]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

In [32]:
df.dropna(subset=metric_cols[:2], inplace=True, how="any")
print(f"{len(df)} experiments after dropping nan")

97 experiments after dropping nan


In [33]:
df.drop_duplicates(subset=param_cols + metric_cols[:2], keep='last', inplace=True)
print(f"{len(df)} experiments after dropping duplicates")

97 experiments after dropping duplicates


In [34]:
df.head()

,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.top_p,params.qa.system_prompt,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
0,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,0.999,direct/format-few.txt,...,5.00,22.0,35.0,8.457143,5.917784,3.0,5.00,6.0,9.00,28.0
1,cb4b874538a73c5774648d414aaddcf69cac56cb,3472a49ed1fb84d89cb1bd950d2f2b2a18c2ae01,rapid-zoon,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,ccot/format-thought-drop.txt,...,286.25,404.0,38.0,246.421053,143.691605,128.0,179.75,209.0,257.50,873.0
2,cb4b874538a73c5774648d414aaddcf69cac56cb,456d70902afc97c9c50d14a66844a3b68bae2bf3,splay-dook,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,4.00,6.0,54.0,6.462963,8.608274,1.0,2.00,3.0,6.75,34.0
3,cb4b874538a73c5774648d414aaddcf69cac56cb,bc72f60fa17d88fcb165fbac7ef813c4b370395d,non-U-shes,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,4.00,41.0,49.0,4.571429,7.302967,1.0,2.00,2.0,4.00,48.0
4,cb4b874538a73c5774648d414aaddcf69cac56cb,e551b58e5ae14d9f8fb622825fcd534e37c5b1a1,oaten-loup,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,29.00,94.0,52.0,35.423077,58.662994,9.0,14.00,24.0,29.00,366.0


In [35]:
def parse_qa_technique(row):
    model = row['params.qa.model']
    if 'r1' in model:
        base_technique = 'REASONING'
    else:
        base_technique = row["params.qa.system_prompt"].split("/")[0].replace(".txt", "").upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [36]:
def remove_ext(filename):
    return filename.split(".", 1)[0]

def parse_instruction(row):
    sp = remove_ext(row["params.qa.system_prompt"]).lower()
    up = remove_ext(row["params.qa.user_prompt_template"]).lower().replace(".txt", "")
    fs = remove_ext(row["params.qa.few_shot_examples"]).lower()
    return f"{sp}:{up}:{fs}"

if len(df):
    df["params.qa.instruction"] = df.apply(parse_instruction, axis=1)

In [37]:
# make temperature 1 decimal
if len(df):
    df["params.qa.temperature"] = df["params.qa.temperature"].round(1)

In [38]:
for col in param_cols:
    values = list(df[col].unique())
    print(f"- {col}: {values}")
    print()

- params.dataset.path: ['bdsaglam/drop-musique-mini']

- params.dataset.name: ['default']

- params.dataset.split: ['train', 'validation']

- params.qa.model: ['llama-3-8b', 'llama-3.3-70b']

- params.qa.temperature: [0.0, 0.5]

- params.qa.top_p: [0.999, 0.95]

- params.qa.system_prompt: ['direct/format-few.txt', 'ccot/format-thought-drop.txt', 'direct/format-few-drop.txt']

- params.qa.user_prompt_template: ['cq.txt']

- params.qa.few_shot_examples: ['manual/direct.json', 'drop/ccot.json', 'drop/direct.json']

- params.qa.n_shot: [0, 5, 1, 3]

- params.qa.n_sc: [1, 5]

- params.run: [1, 2, 3]



In [39]:
df.to_json('exps.jsonl', orient='records', lines=True)

## Setup remaining experiments

In [40]:
def produce_experiment_configs(common_params, varying_params):
    # Generate all possible combinations of parameters
    varying_params = {**common_params, **varying_params}
    keys = varying_params.keys()
    values = varying_params.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [41]:
def produce_all_experiment_configs(common_params: dict, varying_params_list: list[dict]):
    for params in varying_params_list:
        for exp_config in produce_experiment_configs(common_params, params):
            yield exp_config

In [42]:
common_params = {
    "params.dataset.path": ["bdsaglam/drop-musique-mini"],
    "params.dataset.name": ["default"],
    "params.dataset.split": ["validation"],
    "params.qa.n_sc": [
        1,
        5,
    ],
    "params.qa.n_shot": [
        0,
        1,
        3,
        5,
    ],
    "params.run": [
        1,
        2,
        3,
    ],
}

In [43]:
varying_params_list = [
    {
        "params.qa.model": [
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [0.0],
        "params.qa.top_p": [0.999],
        "params.qa.system_prompt": ["direct/format-few-drop.txt"],
        "params.qa.user_prompt_template": ["cq.txt"],
        "params.qa.few_shot_examples": ["drop/direct.json"],
    },
    {
        "params.qa.model": [
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.5,
        ],
        "params.qa.top_p": [
            0.95,
        ],
        "params.qa.system_prompt": ["direct/format-few-drop.txt"],
        "params.qa.user_prompt_template": ["cq.txt"],
        "params.qa.few_shot_examples": ["drop/direct.json"],
    },
    {
        "params.qa.model": [
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.0,
        ],
        "params.qa.top_p": [0.999],
        "params.qa.system_prompt": ["ccot/format-thought-drop.txt"],
        "params.qa.user_prompt_template": ["cq.txt"],
        "params.qa.few_shot_examples": ["drop/ccot.json"],
    },
    {
        "params.qa.model": [
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.5,
        ],
        "params.qa.top_p": [
            0.95,
        ],
        "params.qa.system_prompt": ["ccot/format-thought-drop.txt"],
        "params.qa.user_prompt_template": ["cq.txt"],
        "params.qa.few_shot_examples": ["drop/ccot.json"],
    },
    {
        "params.qa.model": [
            "deepseek-r1-llama-70b",
        ],
        "params.qa.temperature": [
            0.5,
        ],
        "params.qa.top_p": [
            0.95,
        ],
        "params.qa.system_prompt": ["empty.txt"],
        "params.qa.user_prompt_template": ["icq-format-drop.txt"],
        "params.qa.few_shot_examples": ["drop/reasoning.json"],
    },
]

In [44]:
exp_configs = list(produce_all_experiment_configs(common_params, varying_params_list))
target_params = list(exp_configs[0].keys())
print(f"{len(exp_configs)} experiment configurations")
print(target_params)

120 experiment configurations
['params.dataset.path', 'params.dataset.name', 'params.dataset.split', 'params.qa.n_sc', 'params.qa.n_shot', 'params.run', 'params.qa.model', 'params.qa.temperature', 'params.qa.top_p', 'params.qa.system_prompt', 'params.qa.user_prompt_template', 'params.qa.few_shot_examples']


In [45]:
if len(df):
    existing_configs = df[target_params].to_dict(orient="records")
    existing_configs[0]
else:
    existing_configs = []

print("Existing exps:", len(existing_configs))

Existing exps: 97


In [46]:
pairs = []
for exp_config in exp_configs:
    for existing_config in existing_configs:
        if len(sorted(exp_config.items())) != len(sorted(existing_config.items())):
            pairs.append((
                sorted(exp_config.items()), 
                sorted(existing_config.items())
            ))
assert len(pairs) == 0, pairs

In [47]:
# find the missing configurations
missing_configs = [
    dict(kv)
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")

24 missing configurations


In [48]:
def make_command(exp_config):
    lines = ["dvc exp run --queue"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        if isinstance(arg_value, str) and '[' in arg_value:
            arg_value = f'"{arg_value}"'
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [49]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n\n")
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n\n")

## Inspect

In [50]:
df

,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.top_p,params.qa.system_prompt,...,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max,params.qa.technique,params.qa.instruction
0,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,0.999,direct/format-few.txt,...,35.0,8.457143,5.917784,3.0,5.00,6.0,9.00,28.0,DIRECT,direct/format-few:cq:manual/direct
1,cb4b874538a73c5774648d414aaddcf69cac56cb,3472a49ed1fb84d89cb1bd950d2f2b2a18c2ae01,rapid-zoon,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,ccot/format-thought-drop.txt,...,38.0,246.421053,143.691605,128.0,179.75,209.0,257.50,873.0,CCOT-SC(5),ccot/format-thought-drop:cq:drop/ccot
2,cb4b874538a73c5774648d414aaddcf69cac56cb,456d70902afc97c9c50d14a66844a3b68bae2bf3,splay-dook,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,54.0,6.462963,8.608274,1.0,2.00,3.0,6.75,34.0,DIRECT,direct/format-few-drop:cq:drop/direct
3,cb4b874538a73c5774648d414aaddcf69cac56cb,bc72f60fa17d88fcb165fbac7ef813c4b370395d,non-U-shes,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,49.0,4.571429,7.302967,1.0,2.00,2.0,4.00,48.0,DIRECT,direct/format-few-drop:cq:drop/direct
4,cb4b874538a73c5774648d414aaddcf69cac56cb,e551b58e5ae14d9f8fb622825fcd534e37c5b1a1,oaten-loup,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,52.0,35.423077,58.662994,9.0,14.00,24.0,29.00,366.0,DIRECT-SC(5),direct/format-few-drop:cq:drop/direct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,cb4b874538a73c5774648d414aaddcf69cac56cb,94bb530df01a366469600961ba7f192e4a5161ee,glial-ruby,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,50.0,3.240000,2.272821,1.0,2.00,2.0,4.00,10.0,DIRECT,direct/format-few-drop:cq:drop/direct
93,cb4b874538a73c5774648d414aaddcf69cac56cb,3ecfab32d7cd35d648222a7570d39f292085e4aa,acred-tors,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.5,0.950,ccot/format-thought-drop.txt,...,36.0,42.277778,14.149429,25.0,33.50,42.0,47.00,105.0,CCOT,ccot/format-thought-drop:cq:drop/ccot
94,cb4b874538a73c5774648d414aaddcf69cac56cb,c8ba399869f8e1e0035312c10eb6ad427b25e020,minus-flus,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.5,0.950,direct/format-few-drop.txt,...,54.0,4.814815,9.514783,1.0,2.00,2.0,4.00,55.0,DIRECT,direct/format-few-drop:cq:drop/direct
95,cb4b874538a73c5774648d414aaddcf69cac56cb,c754f545eb6239c7a103ad4067ebfa8bda7ccfb9,balmy-ados,bdsaglam/drop-musique-mini,default,validation,llama-3.3-70b,0.0,0.999,direct/format-few-drop.txt,...,54.0,23.240741,32.432185,9.0,14.00,14.0,20.00,244.0,DIRECT-SC(5),direct/format-few-drop:cq:drop/direct
